In [1]:
import morphonet
import numpy as np
import vtkmodules.all as vtk
from skimage import morphology,io,measure,color,filters, img_as_float
import skimage
import nibabel as nib
import matplotlib.pyplot as plt
from scipy.spatial import KDTree

#im = "data/Astec-pm1_intrareg_post_t002.nii"
#im = morphonet.tools.imread(im)
#im=im-morphology.erosion(im)

#morphonet.tools.imsave("data/Astec-pm1_intrareg_post_t002e.nii", im)
#io.imshow(im)
#io.show()

import vtkmodules.all as vtk

def create_base_ico_sphere():
    # Create points
    
    theta = 26.56505117707799 * np.pi / 180
    stheta = np.sin(theta)
    ctheta = np.cos(theta)

    vertices = []
    vertices += [(0.0, 0.0, -1.0)]

    phi = np.pi / 5
    for i in range(1,6): 
        vertices +=[(ctheta * np.cos(phi), ctheta * np.sin(phi), -stheta)]
        phi += 2 * np.pi / 5

    phi = 0
    for i in range(6,11):
        vertices +=[(ctheta * np.cos(phi), ctheta * np.sin(phi), stheta)]
        phi += 2 * np.pi / 5

    vertices += [(0, 0, 1)]

    # Normalize the vertices to form a unit sphere
    vertices = [tuple(np.array(v) / np.linalg.norm(v)) for v in vertices]

    points = vtk.vtkPoints()
    # Add vertices to the mesh
    for v in vertices:
        points.InsertNextPoint(v)

    # Create cells
    icosahedron_faces = [
            (0, 2, 1),
            (0, 3, 2),
            (0, 4, 3),
            (0, 5, 4),
            (0, 1, 5),
            (1, 2, 7),
            (2, 3, 8),
            (3, 4, 9),
            (4, 5, 10),
            (5, 1, 6),
            (1, 7, 6),
            (2, 8, 7),
            (3, 9, 8),
            (4, 10, 9),
            (5, 6, 10),
            (6, 7, 11),
            (7, 8, 11),
            (8, 9, 11),
            (9, 10, 11),
            (10, 6, 11),
        ]

    triangles = vtk.vtkCellArray()
    for i, tri in enumerate(icosahedron_faces):
        triangles.InsertNextCell(3)
        for j in tri:
            triangles.InsertCellPoint(j)

    # Create a polydata object
    mesh = vtk.vtkPolyData()
    mesh.SetPoints(points)
    mesh.SetPolys(triangles)

    return mesh

def create_icosphere(radius: float, centre, subdivisions: int):
    icosphere = create_base_ico_sphere()

    # Loop subdivision
    subdivide = vtk.vtkLoopSubdivisionFilter()
    subdivide.SetNumberOfSubdivisions(subdivisions)
    subdivide.SetInputData(icosphere)
    subdivide.Update()

    icosphere = subdivide.GetOutput()

    points = icosphere.GetPoints()
    #print(points.GetPoint(0))
    vertices = [np.array(points.GetPoint(i)) for i in range(points.GetNumberOfPoints())]
    vertices = [tuple(np.array(v) / np.linalg.norm(v)) for v in vertices]
    vertices = [np.array(points.GetPoint(i))*radius for i in range(points.GetNumberOfPoints())]
    vertices = [np.array(v) + np.array(centre) for v in vertices]
    
    for i in range(points.GetNumberOfPoints()):
        #vertex = points.GetPoint(i)
        #shifted_vertex = [  vertex[0]*radius + centre[0],
        #                    vertex[1]*radius + centre[1],
        #                    vertex[2]*radius + centre[2]]
        #points.SetPoint(i, shifted_vertex)
        points.SetPoint(i, vertices[i])
    icosphere.SetPoints(points)
    
    icosphere.Modified()
    #print(icosphere.GetPoints().GetPoint(0))

    #print(icosphere.GetNumberOfPoints())

    # Write the polydata to an OBJ file
    #writer = vtk.vtkOBJWriter()
    #writer.SetFileName("meshes/icosphere.obj")
    #writer.SetInputData(icosphere)
    #writer.Write()

    return icosphere

def min_distance_index(vertices, surface_pixels):
    min_distance = np.zeros(len(vertices))
    min_index = np.zeros(len(vertices))
    for i in range(len(vertices)):
        distances_from_i = np.sqrt(np.sum((surface_pixels - vertices[i])**2, axis=1))
        min_distance[i] = np.min(distances_from_i)
        min_index[i] = np.argmin(distances_from_i)
    return min_distance,min_index

def min_distance_index2(vertices, surface_pixels_per_vertex):
    min_distance = np.zeros(len(vertices))
    min_index = np.zeros(len(vertices))
    for i in range(len(vertices)):
        distances_from_i = np.sqrt(np.sum((surface_pixels_per_vertex[i] - vertices[i])**2, axis=1))
        min_distance[i] = np.min(distances_from_i)
        min_index[i] = np.argmin(distances_from_i)
    return min_distance,min_index

def perona_malik(image_float):
    # Define the number of iterations and the diffusion coefficient (lambda)
    num_iterations = 10
    diffusion_coefficient = 0.25

    # Apply Perona-Malik anisotropic diffusion
    for _ in range(num_iterations):
        grad_x, grad_y, grad_z = np.gradient(image_float)
        magnitude = np.sqrt(grad_x**2 + grad_y** + grad_z**2)
        c = 1 / (1 + (magnitude / diffusion_coefficient)**2)
        image_float += diffusion_coefficient * (c * np.gradient(c * grad_x) - c * grad_x + 
                                                c * np.gradient(c * grad_y) - c * grad_y + 
                                                c * np.gradient(c * grad_z) - c * grad_z)
    return image_float

def save_meshes(meshes, time, objfile):
    tri_start = 1
    meshes_str = ""
    for i in range(0, len(meshes)):
        mesh_str = "g " + str(time) + "," + str(i) + ",0\n"

        #points = meshes[i].GetPoints()
        for j in range(0, meshes[i].GetNumberOfPoints()):
            p = np.round(np.array(meshes[i].GetPoint(j)), 6)
            mesh_str += "v " + str(p[0]) + " " + str(p[1]) + " " + str(p[2]) + "\n"

        for j in range(0, meshes[i].GetNumberOfCells()):
            tri = meshes[i].GetCell(j)

            p_ids = tri.GetPointIds()
            #print(p_ids.GetNumberOfIds())
            mesh_str += "f " + str(tri_start + p_ids.GetId(0)) + " " + str(tri_start + p_ids.GetId(1)) + " " + str(tri_start + p_ids.GetId(2)) + "\n"

        meshes_str += mesh_str
        tri_start += meshes[i].GetNumberOfPoints()

    # Write the polydata to an OBJ file
    #writer = vtk.vtkOBJWriter()
    #writer.SetFileName("meshes/time02.obj")
    #writer.SetInputData(meshes[0])
    #writer.Write()

    with open(objfile, "w") as file:
        file.write(meshes_str)
    #print(meshes_str)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/Users/joeld/miniconda3/lib/python3.9/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [3]:
cell_meshes = [create_icosphere(10, (0,0,0), 2), create_icosphere(15, (3,0,0), 2)]
save_meshes(cell_meshes, 2, "obj/time01.obj")

In [6]:
im = "nii/embryo1/Astec-Pm1_post_t001.inr"
image = morphonet.tools.imread(im)

#image = filters.sobel(image)

labeled_image = measure.label(image)
regions = measure.regionprops(labeled_image)

print("Number of regions: ", len(regions), "labels: ", np.unique(labeled_image))

 --> Read nii/embryo1/Astec-Pm1_post_t001.inr
Number of regions:  66 labels:  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66]


In [13]:
#from skimage.filters import denoise_tv_bregman
import time

# Iterate through the labeled regions (excluding background)
subdivisions = 2
iterations = 1
#neta = [1 if i % 5 == 0 else 1 for i in range(iterations)]
neta = [1 if i % 5 == 0 else 1 for i in range(iterations)]

nb_regions = 3#len(regions)+1
cell_meshes = []
for cl in range(2, nb_regions):#len(regions)): #regions[1:2]:  # Start from index 1 to exclude background (0)
    #area = region.area
    cell = regions[cl]
    centroid = cell.centroid
    bbox = cell.bbox

    #### Comupte surface pixels
    cell_mask = labeled_image == cl
    cell_surface_mask = cell_mask ^ morphology.erosion(cell_mask, morphology.cube(5))
    
    
    #print(cell_mask.shape, np.sum(cell_mask), np.sum(cell_mask)/(cell_mask.shape[0]*cell_mask.shape[0]*cell_mask.shape[0]))
    #c_b = np.zeros((image.shape))
    #c_b[pixels] = 255
    #print(c_b.shape, image.shape, len(pixels[0]))
    
    #min_cell_mask = lab
    #print(np.sum(c_im))
    #new_im = np.zeros((cell_im.shape))
    #new_im[np.where(cell_im)] = 255
    
    #image_float = img_as_float(gray_image)
    #cell_mask = perona_malik(cell_surface_mask.astype(float))
    """ for slice_index in range(40, cell_mask.shape[0], 20):#cell_im.shape[0]):
        plt.imshow(cell_surface_mask[slice_index], cmap='gray')
        plt.title(f'Slice {slice_index}')
        plt.pause(0.1)  # Pause for a moment to display the slice
        plt.clf()  # Clear the previous plot for the next slice """

    #plt.show()  # Keep the final plot displayed

    pixels = np.where(cell_surface_mask)
    #print(len(pixels[0]))

    #plt.imshow(new_im, cmap='gray')
    #nii = nib.Nifti1Image(new_im, np.eye(4))
    #nib.save(nii, "data/cell_0_t002e.nii")
    #morphonet.tools.imsave("data/cell_0_t002e.nii", new_im)

    s_p_1 = np.where(cell_surface_mask)
    s_p = np.where(cell_surface_mask)
    surface_pixels = np.array([(s_p[0][i], s_p[1][i], s_p[2][i]) for i in range(len(s_p[0]))])
    print(len(s_p[0]), len(s_p_1[0]))
    
    kdtree = KDTree(surface_pixels)
    neighborhood_radius = 3.0
    pixel_neighbours = [kdtree.query_ball_point(p, r=neighborhood_radius) for p in surface_pixels]
    #print(surface_pixels[466], pixel_neighbours[466])
    #surface_pixels[pixel_neighbours[closest_pixel_indices[i]]]

    #print(surface_pixels[1:5])
    #### Compute a spherical mesh and retrieve its vertices
    #centre = centroid
    centre = np.mean(surface_pixels, axis=0)
    #max_radius1 = np.sqrt((bbox[3]-bbox[0])**2 + (bbox[4]-bbox[1])**2 + (bbox[5]-bbox[2])**2)/2
    max_radius = np.max(np.sqrt(np.sum((surface_pixels - centre)**2, axis=1)))
    
    #print(centroid, centre, max_radius1, max_radius)
    icosphere = create_icosphere(max_radius,centre, subdivisions)

    #print(centroid, max_radius*1.2)

    points = icosphere.GetPoints()
    vertices = [points.GetPoint(i) for i in range(points.GetNumberOfPoints())]
    start = time.time()
    distances, closest_pixel_indices = min_distance_index(vertices, surface_pixels)
    end = time.time()
    closest_pixel_indices = closest_pixel_indices.astype(int)
    print("closest pixel to vertex 0: ", closest_pixel_indices[0], end-start)

    #### Iterate the update of the vertex positions until the mesh takes the shape of the cell
    for epoch in range(0, iterations):
        #points = icosphere.GetPoints()
        #vertices = [points.GetPoint(i) for i in range(points.GetNumberOfPoints())]

        #### Compute the distances of mesh vertices to the surface of the cell
        
        surface_pixels_per_vertex = [surface_pixels[pixel_neighbours[closest_pixel_indices[i]]] for i in range(len(vertices))]
        #print(surface_pixels_per_vertex)
        distances, closest_pixel_indices = min_distance_index2(vertices, surface_pixels_per_vertex)
        #distances, closest_pixel_indices = min_distance_index(vertices, surface_pixels)
        closest_pixel_indices = closest_pixel_indices.astype(int)
        #print(closest_pixel_indices[0])

        #distances = [np.min(np.sqrt(np.sum((surface_pixels - p)**2, axis=1))) for p in vertices]
        #closest_pixel_indices = [np.argmin(np.sqrt(np.sum((surface_pixels - p)**2, axis=1))) for p in vertices]

        #### Compute distance gradient for all mesh vertices in the 3d space
        dl = 1 # step

        # grad x
        vertices_x = [np.array(p) + [dl, 0, 0] for p in vertices]
        #distances_x = [np.min(np.sqrt(np.sum((surface_pixels - p)**2, axis=1))) for p in vertices_x]
        distances_x = [np.min(np.sqrt(np.sum((surface_pixels[pixel_neighbours[closest_pixel_indices[i]]] - vertices_x[i])**2, axis=1))) for i in range(len(vertices_x))]
        grad_x = np.array(distances_x) - np.array(distances)

        # grad y
        vertices_y = [np.array(p) + [0, dl, 0] for p in vertices]
        #distances_y = [np.min(np.sqrt(np.sum((surface_pixels - p)**2, axis=1))) for p in vertices_y]
        distances_y = [np.min(np.sqrt(np.sum((surface_pixels[pixel_neighbours[closest_pixel_indices[i]]] - vertices_y[i])**2, axis=1))) for i in range(len(vertices_y))]
        grad_y = np.array(distances_y) - np.array(distances)

        # grad z
        vertices_z = [np.array(p) + [0, 0, dl] for p in vertices]
        #distances_z = [np.min(np.sqrt(np.sum((surface_pixels - p)**2, axis=1))) for p in vertices_z]
        distances_z = [np.min(np.sqrt(np.sum((surface_pixels[pixel_neighbours[closest_pixel_indices[i]]] - vertices_z[i])**2, axis=1))) for i in range(len(vertices_z))]
        grad_z = np.array(distances_z) - np.array(distances)

        grad = [(grad_x[i], grad_y[i], grad_z[i]) for i in range(len(vertices))]    
        
        #### Compute normals and mean curvatures at every mesh vertex
        # Normals
        normals_filter = vtk.vtkPolyDataNormals()
        normals_filter.SetInputData(icosphere)
        normals_filter.ComputePointNormalsOn()
        normals_filter.ComputeCellNormalsOn()
        normals_filter.Update()
        
        # Curvatures
        curvatures_filter = vtk.vtkCurvatures()
        curvatures_filter.SetInputData(icosphere)
        curvatures_filter.SetCurvatureTypeToMean()
        curvatures_filter.Update()
        
        #if (epoch==0):
        normals = [normals_filter.GetOutput().GetPointData().GetNormals().GetTuple(i) for i in range(len(vertices))]
        mean_curvatures = [curvatures_filter.GetOutput().GetPointData().GetScalars().GetTuple1(i) for i in range(len(vertices))]
        
        #### Compute scheme velocities based on distance gradient, normals and curvatures
        velocities = np.array([(np.dot(np.array(grad[i]), np.array(normals[i])) 
                    + 0.5 * mean_curvatures[i]*distances[i]
                    )* 0.5 * np.array(normals[i]) 
                    * (distances[i]/np.max(distances))
                    for i in range(len(vertices))
                ])
        #### Update vertex positions
        vertices = [np.array(vertices[i]) - neta[epoch] * np.array(velocities[i]) for i in range(len(vertices))]
        #radii = [np.sqrt(np.sum((np.array(vertices[i]) - np.array(centre))**2)) for i in range(len(vertices))]

        #print(epoch, np.mean(distances), distances)

        #### Update mesh
        for i in range(points.GetNumberOfPoints()):
            points.SetPoint(i, vertices[i])
        icosphere.SetPoints(points)
        icosphere.Modified()

        print("Cell " + str(cl-1) + "/" + str(nb_regions-1) + ", epoch " + str(epoch) + "/" + str(iterations))

    cell_meshes+=[icosphere]

    

# Write the polydata to an OBJ file
#save_meshes(cell_meshes, 2, "obj/embryo_t01.obj")

#writer = vtk.vtkOBJWriter()
#writer.SetFileName("meshes/icosphere.obj")
#writer.SetInputData(cell_meshes[0])
#print(writer)
#writer.Write()    

165198 165198
closest pixel to vertex 0:  90363 0.7750248908996582
Cell 1/2, epoch 0/1


# Updates to the code that made it work
- First region represents the background, and not any cell. labelled regions indexing starts with 1
- Sphere radius is not given by lengths across dimensions, but by their norm
- Centre is not given by the centroid of the region, but by the centre of mass of the cell surface mask

In [ ]:
im = "data/Astec-pm1_intrareg_post_t002.nii"
im = morphonet.tools.imread(im)

In [51]:
s_p = [(1,1,1), (2, 2, 2)]
vrt = [(0,0,0)]

res = np.sum((np.array(s_p) - vrt)**2, axis=1)

print(res)

[ 3 12]


In [ ]:
alternating_list = [1 if i % 2 == 0 else 5 for i in range(100)]
print(alternating_list)

[1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5]


In [36]:
def compute_gradient_div(mesh, field):
    
    num_vertices = mesh.GetNumberOfPoints()
    num_triangles = mesh.GetNumberOfCells()
    vertices = np.array([np.array(mesh.GetPoint(i)) for i in range(num_vertices)])

    div = np.zeros(field.shape)

    gradient = np.zeros(div.shape + (3,))
    
    for n in range(num_triangles):

        tri_gradient = np.zeros(gradient[0].shape)
        
        cell = mesh.GetCell(n)

        i = cell.GetPointId(0)
        j = cell.GetPointId(1)
        k = cell.GetPointId(2)

        fi = field[i]
        fj = field[j]
        fk = field[k]

        # Calculate edge vectors
        xixj = vertices[j] - vertices[i]
        xixk = vertices[k] - vertices[i]
        
        ijk_normal = np.cross(xixj, xixk)
        ij_normal = np.cross(ijk_normal, xixj)
        ij_normal = np.linalg.norm(xixj) * ij_normal / np.linalg.norm(ij_normal)
        ik_normal = np.cross(ijk_normal, xixk)
        ik_normal = np.linalg.norm(xixk) * ik_normal / np.linalg.norm(ik_normal)

        tri_gradient = ((fj - fi) * ij_normal + (fk - fi) * ik_normal)/np.linalg.norm(ijk_normal)

        gradient[i] += tri_gradient
        gradient[j] += tri_gradient
        gradient[k] += tri_gradient

    div = np.sum(gradient, axis=gradient.shape[-1]-1)
    
    return gradient, div

In [37]:
im = "obj/embryo2/Astec-pm1_intrareg_post_t002.obj"

# Create a VTK OBJ reader
reader = vtk.vtkOBJReader()
reader.SetFileName(im)
reader.Update()
mesh = reader.GetOutput()

num_vertices = mesh.GetNumberOfPoints()
field = np.array([np.ones(3) if i % 5 == 0 else np.zeros(3) for i in range(num_vertices)])

gradient, div = compute_gradient_div(mesh, field)

[[[ 0.01637539  0.25818517 -0.07931502]
  [ 0.01637539  0.25818517 -0.07931502]
  [ 0.01637539  0.25818517 -0.07931502]]

 [[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 ...

 [[ 0.12229828 -0.3556984  -0.18123698]
  [ 0.12229828 -0.3556984  -0.18123698]
  [ 0.12229828 -0.3556984  -0.18123698]]

 [[-0.24864007 -0.23283324  0.00196566]
  [-0.24864007 -0.23283324  0.00196566]
  [-0.24864007 -0.23283324  0.00196566]]

 [[-0.07776656 -0.35327567 -0.11510453]
  [-0.07776656 -0.35327567 -0.11510453]
  [-0.07776656 -0.35327567 -0.11510453]]]


In [126]:
#print(triangles_per_vertex[1], edges[triangles_per_vertex[1][4]])
#print([triangles[k] for k in triangles_per_vertex[1]])

def read_mesh(mesh_file):
    
    reader = vtk.vtkOBJReader()
    reader.SetFileName(mesh_file)
    reader.Update()
    mesh = reader.GetOutput()
    
    return mesh

def compute_triangles_per_vertex(mesh):
    num_vertices = mesh.GetNumberOfPoints()

    triangles_per_vertex = [[] for _ in range(num_vertices)]
    triangles = []

    for i in range(mesh.GetNumberOfCells()):
        triangle = mesh.GetCell(i)
        vertices = [triangle.GetPointId(j) for j in range(3)]

        triangles += [vertices] 
        for vertex in vertices:
            triangles_per_vertex[vertex] += [i]#.append(i)
    return triangles_per_vertex

def compute_vertex_properties(mesh):

    num_vertices = mesh.GetNumberOfPoints()
    triangles_per_vertex = compute_triangles_per_vertex(mesh)

    vertices = np.zeros((num_vertices, 3))
    for i in range(num_vertices):
        vertices[i,:] = mesh.GetPoint(i)
    center = np.mean(vertices, axis=0)
    centered_vertices = vertices - center
    #print(center)
    r = np.linalg.norm(centered_vertices, axis=1)
    phi = np.arccos(centered_vertices[:, 2] / r)
    theta = np.arctan2(centered_vertices[:, 1], centered_vertices[:, 0]) + np.pi

    dA = np.zeros(num_vertices)
    K = np.zeros((num_vertices, 3))
    for i in range(num_vertices):
        xi = i
        tris_xi = triangles_per_vertex[i]
        
        # Compute cotangents
        for j in range(len(triangles_per_vertex[i])):
            tri = triangles[triangles_per_vertex[i][j]]
            
            xj = tri[1] if xi == tri[0] else tri[2] if xi == tri[1] else tri[0]
            xk = tri[2] if xi == tri[0] else tri[0] if xi == tri[1] else tri[1]
            
            """ tris_xj = triangles_per_vertex[xj]
            tris_xixj = [k for k in tris_xi if k in tris_xj]
            
            verts_tris_xixj = list(set(triangles[tris_xixj[0]] + triangles[tris_xixj[1]]))
            AB = [ab for ab in verts_tris_xixj if ab not in [xi,xj]]

            #print(np.array(vertices[xi]))
            Axi = vertices[xi] - vertices[AB[0]]
            Axj = vertices[xj] - vertices[AB[0]]
            Bxi = vertices[xi] - vertices[AB[1]]
            Bxj = vertices[xj] - vertices[AB[1]]
            coTanAxixj = np.dot(Axi, Axj)/np.linalg.norm(np.cross(Axi, Axj))
            coTanBxixj = np.dot(Bxi, Bxj)/np.linalg.norm(np.cross(Bxi, Bxj)) 

            K[i] += (coTanAxixj + coTanBxixj) * np.array(vertices[xi] - vertices[xj])"""

            # Compute Areas
            
            xixj = vertices[xj] - vertices[xi]
            xjxk = vertices[xk] - vertices[xj]
            xkxi = vertices[xi] - vertices[xk]
            xixj_norm = np.linalg.norm(xixj)
            xjxk_norm = np.linalg.norm(xjxk)
            xkxi_norm = np.linalg.norm(xkxi) 

            tri_area = np.linalg.norm(np.cross(xixj,xjxk))/2
            #print(i,j,tri_area, xixj_norm, xjxk_norm, xkxi_norm)
            if xjxk_norm*xjxk_norm > xixj_norm*xixj_norm + xkxi_norm*xkxi_norm:
                dA[i] += tri_area/2
            elif xixj_norm*xixj_norm > xjxk_norm*xjxk_norm + xkxi_norm*xkxi_norm:
                dA[i] += tri_area/4
            elif xkxi_norm*xkxi_norm > xixj_norm*xixj_norm + xjxk_norm*xjxk_norm:
                dA[i] += tri_area/4
            else:
                # Compute Voronoi Area
                IJNormalUnit = np.cross(xixj,np.cross(xixj,xkxi))/np.linalg.norm(np.cross(xixj,np.cross(xixj,xkxi)))
                IKNormalUnit = np.cross(xixj,np.cross(xixj,xkxi))/np.linalg.norm(np.cross(xkxi,np.cross(xixj,xkxi)))
                
                S = (xixj_norm + xjxk_norm + xkxi_norm) / 2
                K_val = np.sqrt(S * (S - xkxi_norm) * (S - xjxk_norm) * (S - xkxi_norm))
                
                radius = xkxi_norm * xjxk_norm * xkxi_norm / (4 * K_val)
                val = np.sqrt(np.abs(radius * radius - xixj_norm*xixj_norm / 4))

                side1 = val * IJNormalUnit
                side2 = val * IKNormalUnit

                dA[i] += (xixj_norm * np.linalg.norm(side1) + xkxi_norm * np.linalg.norm(side2)) / 4
            
        #K[i] = K[i] / (2 * A[i])
        #mean_curvatures = [np.linalg.norm(K[i])/2 for i in range(len(K))]
        #normals = [K[i]/(2*mean_curvatures[i]) for i in range(len(K))]
        #print(i, A[i], normals[i], mean_curvatures[i])
        
    return r, theta, phi, dA 
    #Who What When Where Why -> Journalism
    # What Why How -> Scientific
    # "More is Different" from one -> Anderson, Science (1972)

In [127]:
im = "obj/embryo2/Astec-pm1_intrareg_post_t002.obj"
mesh = read_mesh(im)

r, theta, phi, dA = compute_vertex_properties(mesh)

print(np.min(r), np.max(r), np.min(theta), np.max(theta), np.min(phi), np.max(phi))

203.5435321645555 274.6494428004721 6.240121887746142e-05 6.275027698063944 0.0016550648117854682 3.130467053827382


In [118]:
#### Compute normals and mean curvatures at every mesh vertex
vertices = [np.array(mesh.GetPoint(i)) for i in range(num_vertices)]
# Normals
normals_filter = vtk.vtkPolyDataNormals()
normals_filter.SetInputData(mesh)
normals_filter.ComputePointNormalsOn()
normals_filter.ComputeCellNormalsOn()
normals_filter.Update()

# Curvatures
curvatures_filter = vtk.vtkCurvatures()
curvatures_filter.SetInputData(mesh)
curvatures_filter.SetCurvatureTypeToMean()
curvatures_filter.Update()

#if (epoch==0):
normals = [normals_filter.GetOutput().GetPointData().GetNormals().GetTuple(i) for i in range(len(vertices))]
mean_curvatures = [curvatures_filter.GetOutput().GetPointData().GetScalars().GetTuple1(i) for i in range(len(vertices))]

print(normals)

[(-0.09013095498085022, -0.2602320611476898, -0.9613301157951355), (0.870819628238678, 0.4562283754348755, -0.18310868740081787), (-0.21084600687026978, 0.8058058619499207, -0.553372323513031), (-0.9333726763725281, -0.27115803956985474, -0.23513568937778473), (-0.20453037321567535, -0.8194942474365234, -0.5353469848632812), (0.5990400314331055, -0.7317152619361877, -0.32518270611763), (0.8480797410011292, 0.18353109061717987, 0.4970684349536896), (0.4910593032836914, 0.783000648021698, 0.3817993998527527), (-0.772896409034729, 0.48826444149017334, 0.40525177121162415), (-0.6552448272705078, -0.634407639503479, 0.410099059343338), (-0.043087251484394073, -0.948926568031311, 0.31254100799560547), (-3.024115358130075e-05, 0.0006062954780645669, 0.9999998211860657), (0.44377601146698, 0.4510394334793091, -0.7743553519248962), (-0.17543238401412964, 0.657156229019165, -0.7330546975135803), (0.3348020613193512, 0.8488346934318542, -0.40912988781929016), (-0.5157050490379333, -0.206384450197

In [3]:
def parse_obj_file(obj_file):
    meshes = []
    current_mesh = None
    max_vertex_index = -1

    for line in obj_file:
        print(line)
        line = line.strip()

        if line.startswith('g '):
            # Start of a new mesh
            if current_mesh:
                meshes.append(current_mesh)
            current_mesh = {'vertices': [], 'faces': []}
            max_vertex_index = -1

        elif line.startswith('v '):
            # Vertex line
            if current_mesh:
                parts = line.split()
                if len(parts) == 4:
                    # Parse vertex coordinates
                    vertex = [float(parts[1]), float(parts[2]), float(parts[3])]
                    current_mesh['vertices'].append(vertex)
                    max_vertex_index += 1

        elif line.startswith('f '):
            # Face line
            if current_mesh:
                parts = line.split()
                if len(parts) >= 4:
                    # Parse face indices, offset by the max_vertex_index
                    face_indices = [int(part.split('/')[0]) + max_vertex_index for part in parts[1:]]
                    current_mesh['faces'].append(face_indices)

    if current_mesh:
        meshes.append(current_mesh)

    return meshes


im = "obj/embryo2/cells/Astec-pm1_intrareg_post_t002.obj"
# Open and read your OBJ file
with open(im, 'r') as obj_file:
    meshes = parse_obj_file(obj_file)

# Now, `meshes` is a list where each element is a dictionary representing a mesh.
# Each mesh has a 'vertices' key with a list of vertices and a 'faces' key with a list of faces.

for i, mesh in enumerate(meshes):
    print(f"Mesh {i + 1}:")
    print(f"Vertices: {len(mesh['vertices'])}")
    print(f"Faces: {len(mesh['faces'])}")
    print()

g 1,0,0

v 103.421669 250.466232 238.536835

v 170.49527 301.366669 270.690186

v 82.845245 298.490234 278.962067

v 56.501656 256.982239 281.960815

v 85.651825 199.172623 275.789276

v 146.841263 227.503525 275.646118

v 158.270874 262.208496 321.468567

v 109.272392 293.499023 325.390106

v 55.530586 293.311432 346.473969

v 58.512764 218.676514 345.229248

v 108.491165 209.959244 331.182098

v 87.500374 252.472519 384.281616

v 136.797821 275.50293 250.832779

v 92.784561 285.472107 256.516785

v 125.468941 298.489136 274.384369

v 67.53421 251.459091 251.475204

v 59.383171 290.206787 272.576965

v 89.904854 212.142838 238.560913

v 65.512718 221.768738 274.895355

v 132.928558 233.773621 247.500229

v 116.760483 208.334625 269.104736

v 170.481354 259.233154 268.464752

v 153.687363 299.461456 301.245209

v 102.855011 301.490997 309.160065

v 63.6744 299.462738 311.910095

v 50.516212 275.443085 312.154449

v 51.969032 236.720764 312.49411

v 67.583176 201.711578 311.319733

v 98

In [15]:
import numpy as np
from skimage import io, color, morphology, feature
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time

im = "nii/embryo1/Astec-Pm1_post_t001.inr"
image = morphonet.tools.imread(im)

#image = morphonet.tools.imread(image)
labeled_image = measure.label(image)
regions = measure.regionprops(labeled_image)

cell_surface_masks = []

embryo_mask = labeled_image == 1
embryo_surface_mask = embryo_mask ^ morphology.erosion(embryo_mask, morphology.cube(3))

cell_meshes = []
nb_regions = 5#len(regions)+1
cell_regions_range = range(2, nb_regions)
# Iterate through the labeled regions (excluding background)
for cl in cell_regions_range:#nb_regions):
    print("hello region ", cl)
    start_time_cell = time.time()

    #### Comupte surface pixels
    cell_mask = labeled_image == cl
    cell_surface_mask = cell_mask ^ morphology.erosion(cell_mask, morphology.cube(3))

    cell_surface_masks += [cell_surface_mask]
    
overlap_image = np.sum(cell_surface_masks, axis=0)
print("Sum done")

# Find points where three or more regions overlap
overlap_points = np.where(overlap_image >= 3)
print("Overlaping points found: ", len(overlap_points))

# Visualize the results in 3D
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

#ax.voxels(overlap_image, facecolors='b', edgecolors='k', alpha=0.5)  # Visualize the overlapped structure
ax.scatter(overlap_points[2], overlap_points[1], overlap_points[0], c='r', marker='.', s=50)  # Plot overlap points

ax.set_xlabel('Z-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('X-axis')
ax.set_title('3D Overlapped Image with Overlap Points')

plt.show()

 --> Read nii/embryo1/Astec-Pm1_post_t001.inr


KeyboardInterrupt: 